In [ ]:
## Using qiskit in-built estimator options to test Pauli Twirling. Later learned that Pauli Twirling options are ignored by qiskit during local testing. 

In [1]:
import qiskit
import numpy as np
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import EstimatorV2 as Estimator
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
import qiskit.quantum_info as qi
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime.fake_provider import FakeMumbaiV2

# Import from Qiskit Aer noise module
from qiskit_aer.noise import (NoiseModel, coherent_unitary_error)

In [2]:
def circular_ansatz_mirrored(N, reps=1, fix_2q=False): 
    qc = qiskit.QuantumCircuit(N)
    for _ in range(reps):
        for i in range(N):
            qc.ry(np.pi/2, i)
        for i in range(N):
            qc.rz(np.pi/2, i)
        for i in range(N):
            control = (i-1) % N
            target = i
            qc.cx(control, target)
        for i in range(N):
            qc.ry(np.pi/2, i)
        for i in range(N):
            qc.rz(np.pi/2, i)
        for i in range(N-1, -1, -1):
            control = (i-1) % N
            target = i
            qc.cx(control, target)
    for i in range(N):
        qc.ry(np.pi/2, i)
    for i in range(N):
        qc.rz(np.pi/2, i)
    return qc

def populated_circuit(n):
    qc = qiskit.QuantumCircuit(n)
    qc.h(0)
    for i in range(n - 1):
        qc.cx(i, i + 1)
        if i + 2 < n:
            qc.cz(i + 1, i + 2)
    return qc

In [3]:
num_qubits = 10

np.random.seed(0)
paulis = ["".join(np.random.choice(['I', 'X', 'Y', 'Z'], size=num_qubits)) for _ in range(num_qubits)]

qc = circular_ansatz_mirrored(num_qubits)

#Aer Backend
backend = AerSimulator(method='statevector', device='GPU',seed_simulator = 0)

ideal_estimator = Estimator(mode=backend)
ideal_estimator.options.seed_estimator=0

In [4]:
np.random.seed(0)
coeffs = np.random.random(len(paulis))
observable = SparsePauliOp.from_list(list(zip(paulis, coeffs)))
# observable = SparsePauliOp("Z" * num_qubits)

In [5]:
pm = generate_preset_pass_manager(backend=backend, optimization_level=1,seed_transpiler=0)
isa_circuit = pm.run(qc)
isa_observable = observable.apply_layout(isa_circuit.layout)

In [6]:
job = ideal_estimator.run([(isa_circuit, isa_observable)])

/home/dbharadwaj/.local/lib/python3.10/site-packages/qiskit_ibm_runtime/fake_provider/local_service.py:233: UserWarning: Options {'seed_estimator': 0} have no effect in local testing mode.
  warnings.warn(f"Options {options_copy} have no effect in local testing mode.")


In [7]:
# Get results for the first (and only) PUB
ideal_res = job.result()[0]
print(f">>> Expectation value: {ideal_res.data.evs}")

>>> Expectation value: -0.02461625332245609


## Noisy

In [8]:
# V1

noise_model = NoiseModel()

# # Depolarizing Noise
# cx_depolarizing_prob = 0.1
# noise_model.add_all_qubit_quantum_error(
#     depolarizing_error(cx_depolarizing_prob, 2), ["cx"]
# )

#Coherent Noise
epsilon = 0.1

err_cx = qiskit.QuantumCircuit(2)
err_cx.cx(0,1)
err_cx.p(epsilon, 0)
err_cx.p(epsilon, 1)

err_cx.cx(0,1)
err_cx.p(-epsilon, 0)
err_cx.p(-epsilon, 1)

err_cx = qi.Operator(err_cx)
noise_model.add_all_qubit_quantum_error(
    coherent_unitary_error(err_cx), ["cx"]
)

noisy_backend = AerSimulator(method='statevector', device='GPU',seed_simulator = 0,
                       noise_model=noise_model)

# noisy_backend = FakeMumbaiV2()
noisy_estimator = Estimator(mode=noisy_backend)
(noisy_estimator.options.simulator.noise_model) = noise_model #NOTE: Remove this?

In [9]:
# #V2
# # # noisy_backend = AerSimulator(method='statevector', device='GPU',seed_simulator = 0,
# # #                        noise_model=noise_model)


# service = QiskitRuntimeService()
# noisy_backend = service.least_busy(operational=True, simulator=False,min_num_qubits=num_qubits)
 
# # Setting options during primitive initialization
# noisy_estimator = Estimator(mode=noisy_backend, options={"resilience_level": 2})

In [10]:
# Get results for the first (and only) PUB

pm = generate_preset_pass_manager(backend=noisy_backend, optimization_level=1)
isa_circuit = pm.run(qc)
isa_observable = observable.apply_layout(isa_circuit.layout)

job = noisy_estimator.run([(isa_circuit, isa_observable)])
noisy_res = job.result()[0]

print(f">>> Expectation value: {noisy_res.data.evs}")

>>> Expectation value: -0.02450730416948204


In [11]:
abs_error_diff = abs(noisy_res.data.evs - ideal_res.data.evs)
print(f"Absolute error difference: {abs_error_diff}")

Absolute error difference: 0.00010894915297405064


## Pauli Twirling Mitigation

In [12]:
pt_estimator = Estimator(mode=noisy_backend, options={"resilience_level": 2})

pt_estimator.options.twirling.enable_gates = True
pt_estimator.options.twirling.num_randomizations = 100
pt_estimator.options.twirling.shots_per_randomization = 1000
# pt_estimator.options.simulator.noise_model = noise_model

In [13]:
pm = generate_preset_pass_manager(backend=noisy_backend, optimization_level=1)
isa_circuit = pm.run(qc)
isa_observable = observable.apply_layout(isa_circuit.layout)

In [14]:
job = pt_estimator.run([(isa_circuit, isa_observable)])

/home/dbharadwaj/.local/lib/python3.10/site-packages/qiskit_ibm_runtime/fake_provider/local_service.py:186: UserWarning: The resilience_level option has no effect in local testing mode.
  warnings.warn("The resilience_level option has no effect in local testing mode.")
/home/dbharadwaj/.local/lib/python3.10/site-packages/qiskit_ibm_runtime/fake_provider/local_service.py:233: UserWarning: Options {'twirling': {'enable_gates': True, 'num_randomizations': 100, 'shots_per_randomization': 1000}} have no effect in local testing mode.
  warnings.warn(f"Options {options_copy} have no effect in local testing mode.")


In [15]:
# Get results for the first (and only) PUB
pt_res = job.result()[0]
print(f">>> Expectation value: {pt_res.data.evs}")

>>> Expectation value: -0.02450730416948204


In [16]:
abs_error_diff_pt = abs(pt_res.data.evs - ideal_res.data.evs)
print(f"Absolute error difference: {abs_error_diff_pt}")

Absolute error difference: 0.00010894915297405064


In [17]:
factor_of_difference = abs_error_diff / abs_error_diff_pt
print(f"Factor of Improvement: {factor_of_difference}")

Factor of Improvement: 1.0
